In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.2 --progress-bar off
!pip install -qqq transformers==4.36.2 --progress-bar off
!pip install -qqq einops==0.7.0 --progress-bar off
!pip install -qqq accelerate==0.25.0 --progress-bar off

### Load Model

In [3]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model_name = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = "auto",
    device_map = "auto",
    trust_remote_code = True,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code = True
)

generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.do_sample = True

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Pipeline

In [4]:
from transformers import pipeline, TextStreamer

streamer = TextStreamer(
    tokenizer, 
    skip_prompt = True, 
    skip_special_tokens = True
)

llm = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    return_full_text = True,
    generation_config = generation_config,
    num_return_sequences = 1,
    eos_token_id = tokenizer.eos_token_id,
    pad_token_id = tokenizer.eos_token_id,
    streamer = streamer,
)

2024-02-21 15:39:03.268053: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 15:39:03.268153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 15:39:03.393897: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Prompt

In [5]:
from inspect import cleandoc

SYSTEM_PROMPT = """
You're helpful assistant that always answers truthfully.
""".strip()

def create_prompt(prompt: str, system_prompt: str = SYSTEM_PROMPT) -> str:
    if not system_prompt:
        return cleandoc(
            f"""
            Instruct: {prompt}
            Output:
            """
        )
    
    return cleandoc(
        f"""
        Instruct: {system_prompt} {prompt}
        Output:
        """
    )

In [7]:
prompt = create_prompt("What is the capital of Egpyt?")
print(prompt)

Instruct: You're helpful assistant that always answers truthfully. What is the capital of Egpyt?
Output:


### Testing

In [9]:
output = llm(create_prompt("What is the capital of Egypt? and, what is famous for?"))

 The capital of Egypt is Cairo. It is famous for the Great Pyramids of Giza, the Sphinx, and the Nile River.



In [10]:
output = llm(create_prompt("Write a python function that calculates the squareroot of a multiplication of two numbers."))

 import math

def sqrt_multiply(a, b):
    return math.sqrt(a * b)



In [11]:
output = llm(create_prompt("Write a function in python that checks for palindrom number."))

 def is_palindrome(n):
    n = str(n)
    return n == n[::-1]



In [16]:

tweet = """
I hope that even my worst critics remain on Twitter,
because that is what free speech means
- Elon Musk
"""

prompt = cleandoc(
    f"""
What is the meaning of this tweet? Do sentiment analysis.
Rewrite it in the words of Marcus Aurelius.
```
{tweet}
```
"""
)

output = llm(create_prompt(prompt))

The meaning of this tweet is that free speech is important and should be protected.



In [22]:
output = llm(create_prompt("Naira is faster than Aml and Aml is faster that Rehab, Is Rehab is faster than Naira?"))

 Yes, Rehab is faster than Naira.

